2개로 가기(xgboost, randomforest)
1.이상치 제거
-->1% 클리핑(원래는 각 X 피쳐 속성 다 고려해야하지만 시간이 없으니깐...)
2. 인희님이 feature 만드신거 의미있어보이므로 일단은 feature 만들어놓고 생각
--> feature 만드는건 바로 아래 글 참조
feature 추가해서 만들고, 해당 feature 만드는데 사용되었던 feature은 일단 drop x,
+반드시 통과여부 drop하기******
3. rough하게 준 모델 parameter tuning for each y feature
xgboost and randomforest
xgboost initial 파라미터
xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,colsample_bytree=1, max_depth=7)
randomforest initial 파라미터
RandomForestRegressor(criterion="squared_error", random_state=2022, n_jobs=-1, n_estimators=300, max_depth=150, min_samples_leaf=2, min_samples_split=2)
)

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import random
import os
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,13): # ignore 'ID'
        rmse = mean_squared_error(np.array(gt)[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(np.array(gt)[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [4]:
def lg_nrmse_one(gt, preds):
    # Y_08의 경우는 가중치 1.2 부여. 8 이후부터는 모두 가중치 1.0
    all_nrmse = []
    rmse = mean_squared_error(np.array(gt), preds, squared=False)
    nrmse = rmse/np.mean(np.abs(np.array(gt)))
    all_nrmse.append(nrmse)
    score = 1.0 * np.sum(all_nrmse)
    return score

#얘를 쓸것!!!

In [5]:
train_df = pd.read_csv('/content/drive/MyDrive/LG Aimer/train.csv').drop(columns="ID")
train_x = train_df.filter(regex='X') # Input : X Featrue
train_x=train_x.drop(columns=['X_04','X_23','X_47','X_48'])
train_y = train_df.filter(regex='Y') # Output : Y Feature



#Y_11


In [6]:
train_y11 = train_y['Y_11']
train_y11

0        23.792
1        24.691
2        24.649
3        24.913
4        25.299
          ...  
39602    24.261
39603    23.427
39604    24.301
39605    23.305
39606    24.450
Name: Y_11, Length: 39607, dtype: float64

In [7]:
train_y08=pd.DataFrame(train_y11, index= train_y.index)

#클리핑

In [8]:
from sklearn.model_selection import train_test_split

# -----------------------------------
# 열마다 학습 데이터의 1%, 99% 지점을 확인
p01 = train_x.quantile(0.01)
p99 = train_x.quantile(0.99)

# 1％점 이하의 값은 1%점으로, 99%점 이상의 값은 99%점으로 클리핑
train_x = train_x.clip(p01, p99, axis=1)


#핀 휜 정도 피쳐 만들기

In [9]:
# 커넥터 핀 휨 
train_x['X__21'] = (train_x['X_24'] - train_x['X_12'])
train_x['X__22'] = (train_x['X_25'] - train_x['X_12'])
train_x['X__23'] = (train_x['X_26'] - train_x['X_12'])
train_x['X__24'] = (train_x['X_27'] - train_x['X_12'])
train_x['X__25'] = (train_x['X_28'] - train_x['X_12'])
train_x['X__26'] = (train_x['X_29'] - train_x['X_12'])
#이후, RFE 상에서 혹여나 새로운 변수를 만드는데 기여한 기존 변수들(12, 24,25,26,27,28,29)가 important하게 잡힐 수 있으므로 일단은 놔둡니다.
#또한, 현재 train_x 상에서는 통과여부 변수들은 모두 drop된 상태입니다.


In [10]:
train_x.columns

Index(['X_01', 'X_02', 'X_03', 'X_05', 'X_06', 'X_07', 'X_08', 'X_09', 'X_10',
       'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18', 'X_19',
       'X_20', 'X_21', 'X_22', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29',
       'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_38',
       'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44', 'X_45', 'X_46', 'X_49',
       'X_50', 'X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56', 'X__21',
       'X__22', 'X__23', 'X__24', 'X__25', 'X__26'],
      dtype='object')

RFE 돌리기

#XGBoost

In [11]:
import xgboost as xgb
regressor1= xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,
                           colsample_bytree=1, max_depth=7)


In [12]:
from sklearn.feature_selection import RFE
n_features_to_select=1
rfe1=RFE(regressor1, n_features_to_select=n_features_to_select)

rfe1.fit(train_x, train_y11) 
#다른 데에서 RFE fit할때는 전체 데이터로 하던거 같더라구요
from operator import itemgetter
features = train_x.columns.to_list()
rank_xgb = []
for x,y in (sorted(zip(rfe1.ranking_,features), key=itemgetter(0))):
  print(x,y)
  rank_xgb.append(y)



[01:12:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:12:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:13:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:14:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:14:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:15:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:15:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:16:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

#이후 최적의 경우가 어떨때인지를 본다

In [13]:
score_list=[]
for i in range(len(rank_xgb)):
    cols_forxgb = rank_xgb[:(i+1)]
    train_x_xgb = train_x[cols_forxgb]
    x_train,x_valid, y_train,y_valid = train_test_split(train_x_xgb,train_y11,test_size=0.3) 

    model_xgb = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,colsample_bytree=1, max_depth=7)
    model_xgb.fit(x_train, y_train) ##xgboost error....?
    y_predict = model_xgb.predict(x_valid)
    score = lg_nrmse_one(y_valid, y_predict)
    score_list.append(score)

#여기서 최대인 애 인덱스 찾으면 해당 경우가 최적


[01:34:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:34:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:34:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:34:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:34:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:34:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:34:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:34:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [14]:
score_list

[0.03425845689047604,
 0.03381518247171238,
 0.03392802582801594,
 0.03390667784073375,
 0.03352009701310074,
 0.03348658439038276,
 0.03318007402068007,
 0.03351372012107208,
 0.03309702332978709,
 0.033569982150260226,
 0.03340714317225316,
 0.03292986561206377,
 0.03358414707124314,
 0.03313727498157276,
 0.0332937443354034,
 0.03344617213374809,
 0.033387570453645654,
 0.033222158198131374,
 0.03331127131122226,
 0.03349324835947296,
 0.03298231877331287,
 0.03316382552635222,
 0.03364020625918554,
 0.03331716934163882,
 0.033571359633280735,
 0.033275399188007544,
 0.03344109399648693,
 0.033786259380080313,
 0.033015230185733546,
 0.032971307181564484,
 0.033387684378618374,
 0.03306903420118224,
 0.03323283722988767,
 0.033417801987247704,
 0.03362991330448342,
 0.03340565035357114,
 0.03346432966995056,
 0.033470078730579424,
 0.033503800545306106,
 0.033274554357309194,
 0.033369790349858,
 0.03306211554481423,
 0.03322512288359014,
 0.03333021734734293,
 0.03395128709676967,


In [15]:
tmp = min(score_list)
index = score_list.index(tmp)

print(index)

46


#Randomforest

In [ ]:
regressor2= RandomForestRegressor(criterion="squared_error", random_state=2022, n_jobs=-1, n_estimators=300, max_depth=150, min_samples_leaf=2, min_samples_split=2)
from sklearn.feature_selection import RFE
n_features_to_select=1
rfe2=RFE(regressor2, n_features_to_select=n_features_to_select)

rfe2.fit(train_x, train_y09)
from operator import itemgetter
features = train_x.columns.to_list()
rank_rf = []
for x,y in (sorted(zip(rfe2.ranking_,features), key=itemgetter(0))):
  print(x,y)
  rank_rf.append(y)



1 X_49
2 X_08
3 X_09
4 X_07
5 X_51
6 X_56
7 X_03
8 X_54
9 X_53
10 X_55
11 X_05
12 X_50
13 X_52
14 X_39
15 X_20
16 X_21
17 X_22
18 X_19
19 X_32
20 X_45
21 X_38
22 X_35
23 X__26
24 X_36
25 X_37
26 X_34
27 X_33
28 X_16
29 X_43
30 X_28
31 X_31
32 X_42
33 X_40
34 X_30
35 X__23
36 X_13
37 X_25
38 X_44
39 X_15
40 X_41
41 X__24
42 X_01
43 X__25
44 X_29
45 X_12
46 X_14
47 X__21
48 X_06
49 X_27
50 X_17
51 X_26
52 X__22
53 X_24
54 X_18
55 X_46
56 X_02
57 X_11
58 X_10


In [ ]:
score_list=[]
for i in range(len(rank_rf)):
  cols_forrf = rank_rf[:(i+1)]
  train_x_rf = train_x[cols_forrf]
  x_train,x_valid, y_train,y_valid = train_test_split(train_x_rf,train_y09,test_size=0.3) 
  model_rf =RandomForestRegressor(criterion="squared_error", random_state=2022, n_jobs=-1, n_estimators=300, max_depth=150, min_samples_leaf=2, min_samples_split=2)
  model_rf.fit(x_train,y_train)
  y_predict = model_rf.predict(x_valid)
  score = lg_nrmse_one(y_valid, y_predict)
  score_list.append(score)

#여기서 최대인 애 인덱스 찾으면 해당 경우가 최적


In [ ]:
tmp = min(score_list)
index = score_list.index(tmp)
print(index)

53


여기까지가 feature selection part!
추가로 parameter tuning 후 최종적인 모델 build!